# ASPLOS '23 LeaFTL Experiment

This is a simple experiment that run benchmarks with multi workload and mechanism in Chameleon Cloud.

## Create experiment container

This container provides the following:

- One node of any types ([see all types](https://chameleoncloud.readthedocs.io/en/latest/technical/reservations.html#chameleon-node-types))
- One public IP

### Configuration

Enter your project ID in the code block below.

In [36]:
import chi

chi.use_site("CHI@UC")
chi.set("project_name", "CH-821904")

print(f'Using Project {chi.get("project_name")}')

Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org
Using Project CH-821904


In [37]:
import os

USER = os.getenv('USER')
print(USER)

linjie_ma_rutgers_edu


### Create reservation

Chameleon resources need to be reserved before they can be used. 
We will reserve one bare metal node and one public IP address, for right now.

If you get an error such as "no host available", it may be the case that all of our nodes are reserved. Check the availiablility calendar to see if this is true:
https://chi.uc.chameleoncloud.org/project/leases/calendar/host/

It may take around a minute or so for your lease to become active.

In [38]:
import time
import keystoneauth1, blazarclient
from chi import lease

reservations = []
LEASE_KEY = f"{USER}-asplos23-leaftl"
lease_node_type = "compute_cascadelake_r"

lease_obj=lease.get_lease(LEASE_KEY)

if lease_obj!=None:
    print("lease already created")
    print(lease_obj.node_reservations[0]["id"])
    # print(lease_obj.node_reservations["id"])
else:
    try:
        print(f"Creating lease with name = {LEASE_KEY}...")
        lease.add_fip_reservation(reservations, count=1)
        lease.add_node_reservation(reservations, count=1, node_type=lease_node_type)

        start_date, end_date = lease.lease_duration(hours=6, days=0)

        l = lease.create_lease(
            LEASE_KEY, 
            reservations, 
            start_date=start_date, 
            end_date=end_date
        )
        # l = lease.create_lease(
        #     LEASE_KEY, 
        #     reservations
        # )
        # print(l)
        lease_id = l["id"]
        lease_obj=lease.get_lease(LEASE_KEY)

        print("Waiting for lease to start ...")
        lease_obj.wait()
        print("Lease is now active!")
    #except keystoneauth1.exceptions.http.Unauthorized as e:
        #print("Unauthorized.\nDid set your project name and and run the code in the first cell?")
    except blazarclient.exception.BlazarClientException as e:
        print(f"There is an issue making the reservation. Please check the host calendar.")
        print("https://chi.uc.chameleoncloud.org/project/leases/calendar/host/")
        print(e)
    except Exception as e:
        print("An unexpected error happened.")
        print(e)

lease already created
4cb8a33a-7761-4460-96ea-489a743a2d80


### Provision bare metal node

Next, we will launch the reserved node with an image. 
It will take approximately 10 minutes for the bare metal node to be successfully provisioned. 

This step takes the longest. First, our controller node must configure the requested node, which first sets up a deploy image. This image then downloads and copies the real image onto the hard drive, and the node is configured to reboot to the new OS. 

You can browse the images we offer in our appliance catalog: http://chameleoncloud.org/appliances

In [39]:
from chi import server

image = "CC-Ubuntu20.04"
# print(l["reservations"][0]["id"])

s=server.get_server(LEASE_KEY)

if s!=None:
    print("server already started")
else:
    try:
        s = server.create_server(
            LEASE_KEY, 
            image_name=image,
            reservation_id=lease_obj.node_reservations[0]["id"]
        )
    except Exception as e:
        print("An unexpected error happened.")
        print(e)

    print("Waiting for server to start ...")
    server.wait_for_active(s.id)
    print("Done")

The python binding code in neutronclient is deprecated in favor of OpenstackSDK, please use that as this will be removed in a future release.
The python binding code in neutronclient is deprecated in favor of OpenstackSDK, please use that as this will be removed in a future release.


server already started


By default our node is only connected to a private network and thus not reachable over the internet or via Jupyter here. We need to associate a "Floating IP" to the node, which gives it the public address we reserved.

In [40]:
from chi import lease

floating_ip = lease_obj.get_reserved_floating_ips()[0]
if s.get_floating_ip() != None:
    print("server already get the ip")
else:
    server.associate_floating_ip(s.id, floating_ip_address=floating_ip)

The python binding code in neutronclient is deprecated in favor of OpenstackSDK, please use that as this will be removed in a future release.


server already get the ip


Try to connect to the server through SSH

In [41]:
print(f"Waiting for SSH connectivity on {floating_ip} ...")
timeout = 60 * 2
import socket
import time
# Repeatedly try to connect via SSH.
start_time = time.perf_counter()
while True:
    try:
        with socket.create_connection((floating_ip, 22), timeout=timeout):
            print("Connection successful")
            break
    except OSError as ex:
        time.sleep(10)
        if time.perf_counter() - start_time >= timeout:
            print(f"Timeout: after {timeout} seconds, could not connect via SSH. please wait until SSH is up and ready")

Waiting for SSH connectivity on 192.5.87.186 ...
Connection successful


In [ ]:
## uncomment if your runtime is disconected but, the compile is not finished

# from chi import ssh
# import scripts.session as session

# session_data = session.load()
# floating_ip = session_data["floating_ip"]
# print(f"Waiting for SSH connectivity on {floating_ip} ...")
# timeout = 60 * 2
# import socket
# import time
# # Repeatedly try to connect via SSH.
# start_time = time.perf_counter()
# while True:
#     try:
#         with socket.create_connection((floating_ip, 22), timeout=timeout):
#             print("Connection successful")
#             break
#     except OSError as ex:
#         time.sleep(10)
#         if time.perf_counter() - start_time >= timeout:
#             print(f"Timeout: after {timeout} seconds, could not connect via SSH. please wait until SSH is up and ready")    

Test whether we can run commands through SSH

In [42]:
from chi import ssh

with ssh.Remote(floating_ip) as conn:
    
    # conn.run("lscpu")
    conn.run("uname -a")

Linux linjie-ma-rutgers-edu-asplos23-leaftl-1764128650 5.4.0-198-generic #218-Ubuntu SMP Fri Sep 27 20:18:53 UTC 2024 x86_64 x86_64 x86_64 GNU/Linux


Setup the environment and install all necessary packages

In [43]:
with ssh.Remote(floating_ip) as conn:
    conn.put("setup.sh")
    conn.run("bash ~/setup.sh")

+ sudo apt update




Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists...
Building dependency tree...
Reading state information...
24 packages can be upgraded. Run 'apt list --upgradable' to see them.


+ sudo apt install -y python3-pip unzip




Reading package lists...
Building dependency tree...
Reading state information...
unzip is already the newest version (6.0-25ubuntu1.2).
python3-pip is already the newest version (20.0.2-5ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


+ sudo apt-get install -y libblas-dev liblapack-dev libatlas-base-dev gfortran


...

+ pypy -m pip install scipy==1.2.3
DEPRECATION: pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.


+ pypy -m pip install objgraph==3.5.0
DEPRECATION: pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.


+ pypy -m pip install mmh3==2.5.1 config workflow simpy bitarray==2.9.3 bidict
DEPRECATION: pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.


~


+ popd


Run experiment

In [44]:
with ssh.Remote(floating_ip) as conn:
    conn.put("run_exp.sh")
    conn.run("bash ~/run_exp.sh")

~ ~
[2025-11-26 09:25:57]  Output Directory /home/cc/LeaFTL/wiscsee/leaftl_scripts/raw_results/memory_batch/
[2025-11-26 09:25:57]  Total 21 jobs
[2025-11-26 09:25:58]  Running job 48477: {'start_lineno': 0, 'lineno': 100000, 'trace': '/home/cc/LeaFTL/wiscsee/leaftl_scripts/leaftl_traces/MSR-Cambridge/usr_0.csv', 'cache': 8, 'ftl': 'dftldes', 'write_only': 0, 'ncq_depth': 1, 'page_size': 4096}
[2025-11-26 09:25:58]  pypy run_ftl -sl 0 -l 100000 -t /home/cc/LeaFTL/wiscsee/leaftl_scripts/leaftl_traces/MSR-Cambridge/usr_0.csv -c 8 -f dftldes -wo 0 -q 1 -p 4096 -o /home/cc/LeaFTL/wiscsee/leaftl_scripts/raw_results/memory_batch/0 >> /home/cc/LeaFTL/wiscsee/leaftl_scripts/raw_results/memory_batch/0_stdout.txt
[2025-11-26 09:25:58]  Running job 48479: {'start_lineno': 0, 'lineno': 100000, 'trace': '/home/cc/LeaFTL/wiscsee/leaftl_scripts/leaftl_traces/MSR-Cambridge/prn_0.csv', 'cache': 8, 'ftl': 'dftldes', 'write_only': 0, 'ncq_depth': 1, 'page_size': 4096}
[2025-11-26 09:25:58]  pypy run_ftl 

/home/cc/LeaFTL/pypy2.7-v7.3.9-linux64/site-packages/bidict/compat.py:33: UserWarning: Python 2 support will be dropped in a future release.
  warn('Python 2 support will be dropped in a future release.')
/home/cc/LeaFTL/pypy2.7-v7.3.9-linux64/site-packages/bidict/compat.py:33: UserWarning: Python 2 support will be dropped in a future release.
  warn('Python 2 support will be dropped in a future release.')
/home/cc/LeaFTL/pypy2.7-v7.3.9-linux64/site-packages/bidict/compat.py:33: UserWarning: Python 2 support will be dropped in a future release.
  warn('Python 2 support will be dropped in a future release.')
/home/cc/LeaFTL/pypy2.7-v7.3.9-linux64/site-packages/bidict/compat.py:33: UserWarning: Python 2 support will be dropped in a future release.
  warn('Python 2 support will be dropped in a future release.')
/home/cc/LeaFTL/pypy2.7-v7.3.9-linux64/site-packages/bidict/compat.py:33: UserWarning: Python 2 support will be dropped in a future release.
  warn('Python 2 support will be droppe

pypy run_ftl -sl 0 -l 100000 -t /home/cc/LeaFTL/wiscsee/leaftl_scripts/leaftl_traces/FIU/mail/cheetah.cs.fiu.edu-110108-113008.1.blkparse -c 8 -f sftl -wo 0 -q 1 -p 4096 -o /home/cc/LeaFTL/wiscsee/leaftl_scripts/raw_results/memory_batch/13 >> /home/cc/LeaFTL/wiscsee/leaftl_scripts/raw_results/memory_batch/13_stdout.txt
[2025-11-26 09:25:58]  Running job 48504: {'start_lineno': 0, 'lineno': 100000, 'trace': '/home/cc/LeaFTL/wiscsee/leaftl_scripts/leaftl_traces/MSR-Cambridge/usr_0.csv', 'cache': 8, 'ftl': 'learnedftl', 'write_only': 0, 'ncq_depth': 1, 'page_size': 4096}
[2025-11-26 09:25:58]  pypy run_ftl -sl 0 -l 100000 -t /home/cc/LeaFTL/wiscsee/leaftl_scripts/leaftl_traces/MSR-Cambridge/usr_0.csv -c 8 -f learnedftl -wo 0 -q 1 -p 4096 -o /home/cc/LeaFTL/wiscsee/leaftl_scripts/raw_results/memory_batch/14 >> /home/cc/LeaFTL/wiscsee/leaftl_scripts/raw_results/memory_batch/14_stdout.txt
[2025-11-26 09:25:58]  Running job 48507: {'start_lineno': 0, 'lineno': 100000, 'trace': '/home/cc/LeaFT

Get the graph back to local

In [45]:
with ssh.Remote(floating_ip) as conn:
    conn.get("LeaFTL/wiscsee/leaftl_scripts/plots/memory.png")